# Visual Question Answering — End-to-End Pipeline

**Bài toán:** Cho ảnh + câu hỏi → sinh câu trả lời bằng LSTM-Decoder.

**4 kiến trúc:**

| Model | CNN Encoder | Attention |
|-------|-------------|----------|
| A | Scratch CNN | No |
| B | Pretrained ResNet101 | No |
| C | Scratch CNN | Bahdanau |
| D | Pretrained ResNet101 | Bahdanau |

**Pipeline:**
1. Clone repo + cài đặt dependencies
2. Tải dữ liệu VQA 2.0 từ Kaggle
3. Build vocab (questions + answers)
4. Train 4 models (A, B, C, D)
5. Plot training curves
6. Evaluate từng model (VQA Accuracy, Exact Match, BLEU-1/2/3/4, METEOR)
7. So sánh 4 models side-by-side
8. Inference trên sample
9. Attention Visualization (Model C, D)

---
## Step 0 — Environment Setup

- Kiểm tra GPU
- Clone repository từ GitHub
- Cài đặt dependencies

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Clone repository
!git clone https://github.com/Anakonkai01/new_vqa.git
%cd new_vqa

# Checkout branch (thay đổi nếu cần)
!git checkout experiment/new

In [ ]:
# Cài đặt dependencies
!pip install -q nltk tqdm matplotlib Pillow

import nltk
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

---
## Step 1 — Download VQA 2.0 Data từ Kaggle

Tải 3 datasets:
- **vqa-20-images**: COCO train2014 images
- **vqa-2-0-val2014**: COCO val2014 images
- **vqa2-0-data-json**: VQA 2.0 question + annotation JSON files

> **Note:** Cần cấu hình Kaggle API key trước (upload `kaggle.json` hoặc set biến môi trường).

In [ ]:
# Nếu chưa có kaggle.json, upload nó:
# from google.colab import files
# files.upload()  # upload kaggle.json
# !mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

!pip install -q kaggle

In [ ]:
# Tải dữ liệu từ Kaggle
!kaggle datasets download -d bishoyabdelmassieh/vqa-20-images -p datasets --unzip
!kaggle datasets download -d hongnhnnguyntrn/vqa-2-0-val2014 -p datasets --unzip
!kaggle datasets download -d hongnhnnguyntrn/vqa2-0-data-json -p datasets --unzip

In [ ]:
# Kiểm tra cấu trúc dataset đã tải
import os
print("Downloaded files:")
for root, dirs, files in os.walk('datasets'):
    level = root.replace('datasets', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    if level < 2:  # chỉ hiện 2 levels đầu
        subindent = ' ' * 2 * (level + 1)
        for f in files[:5]:
            print(f"{subindent}{f}")
        if len(files) > 5:
            print(f"{subindent}... ({len(files)} files total)")

### Sắp xếp dữ liệu vào đúng cấu trúc thư mục project

Project yêu cầu cấu trúc:
```
data/raw/images/train2014/   ← COCO train images
data/raw/images/val2014/     ← COCO val images  
data/raw/vqa_json/           ← VQA 2.0 JSON files
data/processed/              ← vocab files (sẽ được tạo ở step sau)
```

> **Quan trọng:** Cell dưới sẽ tạo symlinks/move dữ liệu vào đúng vị trí. Hãy kiểm tra output của cell trên để xác nhận đường dẫn chính xác, nếu cấu trúc Kaggle khác thì sửa lại cell dưới.

In [ ]:
import os, glob, shutil

# Tạo thư mục đích
os.makedirs('data/raw/images', exist_ok=True)
os.makedirs('data/raw/vqa_json', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)
os.makedirs('checkpoints', exist_ok=True)

# ── Helper: tìm thư mục chứa COCO images ─────────────────────────────
def find_coco_dir(base, split):
    """Tìm thư mục chứa ảnh COCO_<split>_*.jpg trong base."""
    for root, dirs, files in os.walk(base):
        for f in files:
            if f.startswith(f'COCO_{split}_') and f.endswith('.jpg'):
                return root
    return None

# ── Symlink train2014 images ──────────────────────────────────────────
train_dir = find_coco_dir('datasets', 'train2014')
if train_dir and not os.path.exists('data/raw/images/train2014'):
    os.symlink(os.path.abspath(train_dir), 'data/raw/images/train2014')
    print(f"Linked train2014: {train_dir} -> data/raw/images/train2014")
elif os.path.exists('data/raw/images/train2014'):
    print("train2014 already exists.")
else:
    print("WARNING: Could not find train2014 images in datasets/")

# ── Symlink val2014 images ────────────────────────────────────────────
val_dir = find_coco_dir('datasets', 'val2014')
if val_dir and not os.path.exists('data/raw/images/val2014'):
    os.symlink(os.path.abspath(val_dir), 'data/raw/images/val2014')
    print(f"Linked val2014: {val_dir} -> data/raw/images/val2014")
elif os.path.exists('data/raw/images/val2014'):
    print("val2014 already exists.")
else:
    print("WARNING: Could not find val2014 images in datasets/")

# ── Copy VQA JSON files ───────────────────────────────────────────────
json_patterns = [
    'v2_OpenEnded_mscoco_train2014_questions.json',
    'v2_OpenEnded_mscoco_val2014_questions.json',
    'v2_mscoco_train2014_annotations.json',
    'v2_mscoco_val2014_annotations.json',
]
for jname in json_patterns:
    dst = f'data/raw/vqa_json/{jname}'
    if os.path.exists(dst):
        print(f"  Already exists: {dst}")
        continue
    # Tìm file trong datasets/
    matches = glob.glob(f'datasets/**/{jname}', recursive=True)
    if matches:
        shutil.copy2(matches[0], dst)
        print(f"  Copied: {matches[0]} -> {dst}")
    else:
        print(f"  WARNING: {jname} not found in datasets/")

# ── Verify ────────────────────────────────────────────────────────────
print("\n--- Verification ---")
for p in ['data/raw/images/train2014', 'data/raw/images/val2014']:
    if os.path.exists(p):
        n = len(os.listdir(p))
        print(f"  {p}: {n:,} files")
    else:
        print(f"  MISSING: {p}")
for p in json_patterns:
    full = f'data/raw/vqa_json/{p}'
    sz = os.path.getsize(full) / 1e6 if os.path.exists(full) else 0
    print(f"  {full}: {sz:.1f} MB" if sz > 0 else f"  MISSING: {full}")

---
## Step 2 — Build Vocabulary

Xây dựng:
- **Question vocabulary**: các từ xuất hiện >= 3 lần trong training questions
- **Answer vocabulary**: các câu trả lời xuất hiện >= 5 lần

Output:
- `data/processed/vocab_questions.json`
- `data/processed/vocab_answers.json`

In [ ]:
!python src/scripts/1_build_vocab.py

In [ ]:
# Kiểm tra vocab đã tạo
import json

with open('data/processed/vocab_questions.json') as f:
    vq = json.load(f)
with open('data/processed/vocab_answers.json') as f:
    va = json.load(f)

print(f"Question vocab size: {len(vq['word2idx'])}")
print(f"Answer vocab size  : {len(va['word2idx'])}")
print(f"\nSample question words: {list(vq['word2idx'].keys())[:15]}")
print(f"Sample answer words  : {list(va['word2idx'].keys())[:15]}")

---
## Step 3 — Train All 4 Models

Train lần lượt 4 kiến trúc:

| Model | Encoder | Attention | Đặc điểm |
|-------|---------|-----------|----------|
| A | Scratch CNN (5 conv blocks) | No | Baseline, train từ đầu |
| B | ResNet101 (pretrained, frozen) | No | Transfer learning |
| C | Scratch CNN Spatial (7×7=49 regions) | Bahdanau | Attention trên scratch features |
| D | ResNet101 Spatial (pretrained, frozen) | Bahdanau | Attention trên pretrained features |

**Hyperparameters:**
- `embed_size=512`, `hidden_size=1024`, `num_layers=2`
- `lr=1e-3`, `batch_size=128`, `epochs=10`
- AMP (mixed precision) tự bật trên GPU
- ReduceLROnPlateau (factor=0.5, patience=2)
- Gradient clipping (max_norm=5.0)

**Output mỗi model:**
- `checkpoints/model_X_epoch{1..10}.pth` — per-epoch checkpoint
- `checkpoints/model_X_best.pth` — checkpoint tốt nhất (lowest val loss)
- `checkpoints/model_X_resume.pth` — full checkpoint để resume
- `checkpoints/history_model_X.json` — train/val loss history

In [ ]:
# Train Model A: Scratch CNN, No Attention
!python src/train.py --model A --epochs 10 --lr 1e-3 --batch_size 128

In [ ]:
# Train Model B: ResNet101 (pretrained, frozen), No Attention
!python src/train.py --model B --epochs 10 --lr 1e-3 --batch_size 128

In [ ]:
# Train Model C: Scratch CNN Spatial, Bahdanau Attention
!python src/train.py --model C --epochs 10 --lr 1e-3 --batch_size 128

In [ ]:
# Train Model D: ResNet101 Spatial (pretrained, frozen), Bahdanau Attention
!python src/train.py --model D --epochs 10 --lr 1e-3 --batch_size 128

In [ ]:
# Kiểm tra checkpoints đã lưu
import os
print("Saved checkpoints:")
for f in sorted(os.listdir('checkpoints')):
    sz = os.path.getsize(f'checkpoints/{f}') / 1e6
    print(f"  {f:45s} {sz:8.1f} MB")

### (Optional) Fine-tuning Model B, D với CNN unfreeze

Sau khi train 10 epochs với ResNet frozen, có thể tiếp tục fine-tune bằng cách:
- Unfreeze layer3 + layer4 của ResNet
- Dùng learning rate nhỏ hơn cho backbone (`cnn_lr_factor=0.1`)

Điều này giúp ResNet adapt features cho VQA thay vì chỉ dùng ImageNet features.

In [ ]:
# (Optional) Fine-tune Model B: resume + unfreeze backbone
# !python src/train.py --model B --epochs 5 --lr 5e-4 --batch_size 128 \
#     --resume checkpoints/model_b_resume.pth --finetune_cnn --cnn_lr_factor 0.1

# (Optional) Fine-tune Model D: resume + unfreeze backbone
# !python src/train.py --model D --epochs 5 --lr 5e-4 --batch_size 128 \
#     --resume checkpoints/model_d_resume.pth --finetune_cnn --cnn_lr_factor 0.1

### (Optional) Scheduled Sampling

Giảm **exposure bias** (mismatch giữa teacher forcing training vs autoregressive inference):
- Epoch đầu: chủ yếu dùng ground-truth token (epsilon ≈ 1)
- Epoch cuối: chủ yếu dùng model's own prediction (epsilon → 0)
- Schedule: `epsilon = k / (k + exp(epoch/k))`

In [ ]:
# (Optional) Train với Scheduled Sampling
# !python src/train.py --model A --epochs 10 --lr 1e-3 --batch_size 128 --scheduled_sampling --ss_k 5

---
## Step 4 — Plot Training Curves

So sánh train/val loss của 4 models qua các epochs.

Output: `checkpoints/training_curves.png`

In [ ]:
!python src/plot_curves.py --models A,B,C,D --output checkpoints/training_curves.png

In [ ]:
# Hiển thị training curves
from IPython.display import Image, display
display(Image(filename='checkpoints/training_curves.png'))

---
## Step 5 — Evaluate từng Model

Đánh giá mỗi model trên **validation set** với các metrics:
- **VQA Accuracy**: `min(matching_annotations / 3, 1.0)` — official VQA metric
- **Exact Match**: prediction == ground truth (strict)
- **BLEU-1, BLEU-2, BLEU-3, BLEU-4**: n-gram overlap
- **METEOR**: synonym-aware matching

In [ ]:
# Evaluate Model A
!python src/evaluate.py --model_type A

In [ ]:
# Evaluate Model B
!python src/evaluate.py --model_type B

In [ ]:
# Evaluate Model C
!python src/evaluate.py --model_type C

In [ ]:
# Evaluate Model D
!python src/evaluate.py --model_type D

### (Optional) Evaluate với Beam Search

Thay vì greedy decode (chọn token xác suất cao nhất), beam search giữ top-k candidates tại mỗi bước để tìm sequence tốt hơn.

In [ ]:
# (Optional) Evaluate với beam search width=3
# !python src/evaluate.py --model_type D --beam_width 3

---
## Step 6 — So sánh 4 Models (Comparison Table)

So sánh tất cả models trên cùng validation set, in bảng side-by-side.

Đây là phần **đánh giá chính** của bài — so sánh:
- Scratch vs Pretrained (A vs B, C vs D)
- No Attention vs Attention (A vs C, B vs D)

In [ ]:
!python src/compare.py --models A,B,C,D --epoch 10

---
## Step 7 — Single-Sample Inference

Chạy inference trên 1 sample cụ thể để xem model sinh câu trả lời như thế nào.

Script `inference.py` mặc định chạy model A trên sample đầu tiên. Có thể sửa trực tiếp trong code nếu muốn đổi model/sample.

In [ ]:
!python src/inference.py

---
## Step 8 — Attention Visualization (Model C, D)

Trực quan hóa cơ chế attention:
- Với mỗi token được sinh ra, hiển thị **heatmap** trên ảnh gốc cho thấy vùng nào model đang "nhìn vào"
- Attention weights `alpha` có shape `(49,)` → reshape thành `7×7` → upsample lên `224×224`

Output: `checkpoints/attn_model_c.png`, `checkpoints/attn_model_d.png`

In [ ]:
# Attention visualization — Model C
!python src/visualize.py --model_type C --sample_idx 0

In [ ]:
# Attention visualization — Model D
!python src/visualize.py --model_type D --sample_idx 0

In [ ]:
# Hiển thị attention maps
from IPython.display import Image, display
import os

for mt in ['c', 'd']:
    path = f'checkpoints/attn_model_{mt}.png'
    if os.path.exists(path):
        print(f"\n--- Model {mt.upper()} Attention ---")
        display(Image(filename=path))
    else:
        print(f"Not found: {path}")

---
## Summary

| Step | Script | Output |
|------|--------|--------|
| Build Vocab | `src/scripts/1_build_vocab.py` | `data/processed/vocab_*.json` |
| Train | `src/train.py --model X` | `checkpoints/model_X_epoch*.pth`, `history_model_X.json` |
| Plot Curves | `src/plot_curves.py` | `checkpoints/training_curves.png` |
| Evaluate | `src/evaluate.py --model_type X` | Printed metrics |
| Compare | `src/compare.py` | Side-by-side comparison table |
| Inference | `src/inference.py` | Question + Predicted answer |
| Attention | `src/visualize.py --model_type C/D` | `checkpoints/attn_model_*.png` |

### Kiến trúc

```
Image ──> CNN Encoder ──> img_feature ──┐
                                        ├── Hadamard Fusion ──> h_0 ──> LSTM Decoder ──> Answer tokens
Question ──> LSTM Encoder ──> q_feature ─┘         ↑
                                          (Model C,D: Bahdanau Attention
                                           attends over 49 spatial regions)
```

### Kết quả đánh giá

_(Xem output Step 6 ở trên để điền bảng kết quả)_